In [14]:
cik = 1137774
url="https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK={}&type=10-k&dateb=&owner=exclude&count=40".format(cik)
import requests
from bs4 import BeautifulSoup
import pandas as pd

response=requests.get(url)
result_page=BeautifulSoup(response.content,'lxml')
exturl = result_page.find_all('a',id='documentsbutton')
list_=list()
for ext in exturl:
    link = "https://www.sec.gov/"+ ext.get('href')
    list_.append(link)
dict_k = dict()

for i in list_:
    page = BeautifulSoup(requests.get(i).content, 'lxml')
    if page.find('strong').text == "Form 10-K":
        year = page.find_all('div',class_="info")[3].text
        test="https://www.sec.gov/"+ page.find_all('td')[2].find('a').get('href')
        if "htm" in test:
    #         print(test)
            dict_k[year]=test

In [33]:
new_dict=dict()
for key, value in dict_k.items():
    try:
        dfs = pd.read_html(value, match="Payments Due by Period") 
        new_row = list()
        row= dfs[0]
        last = row.dropna(thresh=5).iloc[-1]
        for i in last:
            try:
                isinstance(float(i), float)
                new_row.append(float(i))
                data=pd.DataFrame(new_row).dropna()
                max_=data.max()
                data1=data.drop(data.idxmax()) 

            except ValueError:
                pass
        new_dict[key]=data1.iloc[0]
    except:
        pass
print(new_dict)


{'2016-12-31': 0    65366.0
Name: 4, dtype: float64, '2015-12-31': 0    59982.0
Name: 4, dtype: float64, '2014-12-31': 0    73277.0
Name: 4, dtype: float64, '2013-12-31': 0    70007.0
Name: 4, dtype: float64, '2012-12-31': 0    65457.0
Name: 4, dtype: float64, '2011-12-31': 0    64247.0
Name: 4, dtype: float64, '2010-12-31': 0    57992.0
Name: 4, dtype: float64, '2009-12-31': 0    60511.0
Name: 3, dtype: float64, '2008-12-31': 0    74926.0
Name: 3, dtype: float64, '2007-12-31': 0    94881.0
Name: 3, dtype: float64, '2006-12-31': 0    46981.0
Name: 3, dtype: float64, '2005-12-31': 0    13306.0
Name: 3, dtype: float64, '2004-12-31': 0    5079.0
Name: 3, dtype: float64, '2003-12-31': 0    8482.0
Name: 3, dtype: float64}


In [54]:
df_final=pd.DataFrame(new_dict,index=(['Prudential'])) 
df_final

,2003-12-31,2004-12-31,2005-12-31,2006-12-31,2007-12-31,2008-12-31,2009-12-31,2010-12-31,2011-12-31,2012-12-31,2013-12-31,2014-12-31,2015-12-31,2016-12-31
Prudential,8482.0,5079.0,13306.0,46981.0,94881.0,74926.0,60511.0,57992.0,64247.0,65457.0,70007.0,73277.0,59982.0,65366.0
